In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Lecture 10 #

## Functions with multiple arguments and review of tb.apply()

In [ ]:
ages = Table().with_columns(
    'Person', make_array('A', 'B', 'C', 'D'),
    'Age', make_array(23, 110, 28, 102)
)
ages

In [ ]:
# define the cut_off function that takes two arguments
def cut_off(z, cut_off_value = 100):
    return min(z, cut_off_value)

In [ ]:
# get its type
type(cut_off)

In [ ]:
# try out the function
cut_off(3)

In [ ]:
cut_off(107)

In [ ]:
cut_off(57, 50)

In [ ]:
# apply the function to a table
cut_age_array = ages.apply(cut_off, 'Age')
cut_age_array

In [ ]:
ages.with_column('Cut off ages', cut_age_array)

In [ ]:
# define another derived function called cut_off_50
def cut_off_50(x):
    return cut_off(x, 50)

In [ ]:
cut_off_50(67)

In [ ]:
ages.apply(cut_off_50, 'Age')

## Prediction example ##

To practice applying functions to data, let's look at data collected by Francis Galton.


In [ ]:
galton = Table.read_table('galton.csv')

In [ ]:
#Each row corresponds to one adult child
#family = family indicator
#father height (inches) 
#mother height (inches) 
#"midparent height"= weighted average of parents' heights
#children= # of children in the family
#childNum = child's birth rank (1 = oldest)
#gender
#height (inches)
galton

In [ ]:
# get only the columns we need
heights = galton.select(3, 7).relabeled(0, 'MidParent').relabeled(1, 'Child')

In [ ]:
heights

In [ ]:
# Side note:  overlapping histogram 
heights.hist(bins=20, unit='inches')

In [ ]:
# create a scatter plot of the data
heights.scatter('MidParent', 'Child')

In [ ]:
# add vertical lines in a particular range of values
heights.scatter('MidParent', 'Child')
plots.plot([67.5, 67.5], [50, 85], color='red', lw=2)
plots.plot([68.5, 68.5], [50, 85], color='red', lw=2);

In [ ]:
# calculate mean of childrens' heights for parents whose heights are in a particular range
nearby = heights.where('MidParent', are.between(67.5, 68.5))
nearby.column('Child').mean()

In [ ]:
# plot the mean child height on the scatter plot
heights.scatter('MidParent', 'Child')
plots.plot([67.5, 67.5], [50, 85], color='red', lw=2)
plots.plot([68.5, 68.5], [50, 85], color='red', lw=2)
plots.scatter(68, 66.24, color='gold', s=50);

In [ ]:
# write a function to predict a child's height based on their (average) parents' heights that fall in a particular range
def predict_child(h):
    nearby = heights.where('MidParent', are.between(h - 0.5, h + 0.5))
    return nearby.column('Child').mean()

predict_child(68)

In [ ]:
# try to make some predictions
predict_child(70)

In [ ]:
predict_child(72)

In [ ]:
# predict all values in our data set
predicted = heights.apply(predict_child, 'MidParent')
predicted

In [ ]:
# add predictions to our heights table and plot the results
heights = heights.with_column('Predicted child', predicted)
heights

In [ ]:
heights.scatter('MidParent')

## Apply with Multiple Columns ##

If we have a function that takes two arguments, we can apply it to values in two columns. 

Let's examine the range of temperatures that occur on different days.


In [ ]:
temperatures = Table.read_table('temperatures.csv')
temperatures

In [ ]:
# plot the high and the low temperature as a function of the day
temperatures.plot('Day')

In [ ]:
# plot the high and the low temperature overlapping histograms
temperatures.select('Low', 'High').hist(bins=np.arange(30, 105, 5))

In [ ]:
# create a scatter plot of the high and the low temperature
temperatures.scatter('Low', 'High')

In [ ]:
# Difference between high temp and low temp
def difference(x, y):
    return x-y

difference(65, 54)

In [ ]:
# calculate the spread of temperatures on each day and add it to our table
daily_spread = temperatures.apply(difference, 'High', 'Low')
temperatures = temperatures.with_column('Spread', daily_spread)
temperatures

In [ ]:
# create a histogram of the spreads of temperatures
temperatures.hist('Spread', bins=np.arange(0, 40, 4))

In [ ]:
# calculate the proportion of days that have more than a 20 degree spread in temperatures
temperatures.where('Spread', are.above(20)).num_rows / temperatures.num_rows

## Grouping by Category ##

Previously we've seen how we can use the `tb.group()` method to calculate the number of values in each group. We will now also use to to apply a function separately to each group. 


In [ ]:
all_cones = Table.read_table('cones.csv')
all_cones

In [ ]:
# remove the Color column for now
cones = all_cones.drop('Color').exclude(5)
cones

In [ ]:
# count how many cones there are of each Flavor
cones.group('Flavor')

In [ ]:
# Calculate the minimum price for each Flavor
cones.group('Flavor', min)

In [ ]:
# Get the average price for each Flavor
cones.group('Flavor', np.average)

In [ ]:
# Get a list of prices for each Flavor
cones.group('Flavor', list)

In [ ]:
# User-defined function
def spread(arr):
    return max(arr) - min(arr)

spread(make_array(7, 10, 2))

In [ ]:
# Use your own function in .group
cones.group('Flavor', spread)

## Grouping by multiple columns

We can also group by multiple columns. This will allow give us separate values for each combination of values in the columns we group by. 

In [ ]:
cones

In [ ]:
all_cones

In [ ]:
# Count how many items have a particular Flavor and Color combination
all_cones.group(['Flavor', 'Color'])

In [ ]:
# Take the average price for each particular Flavor and Color combination
all_cones.group(['Flavor', 'Color'], np.average)

## Example - NBA salaries ##

In [ ]:
nba = Table.read_table('nba_salaries.csv').relabeled(3, 'SALARY')
nba

In [ ]:
# Total salary paid by each team, highest first
nba.select('TEAM', 'SALARY').group('TEAM', sum).sort('SALARY sum', descending=True)

In [ ]:
# what happens if we don't select only numerical columns first?
nba.group('TEAM', sum)

In [ ]:
# Average salary paid for each position
nba.select('POSITION', 'SALARY').group('POSITION', np.average)

In [ ]:
# For each team, average salary paid for each position
nba.drop('PLAYER').group(['TEAM', 'POSITION'], np.average)